In [18]:
import numpy as np
from task1 import *

## Our Algorithm

The core idea behind our algorithm is using a binary search tree, where each node has a boolean condition property and each leaf node has a value property. When making a prediction, if the condition property is true for some given input $\vec{x}$, then it will traverse down the right subtree. Otherwise, it will traverse down the left subtree. These conditions will always be of the form
$$
\vec{x}_f \leq \text{value}
$$
where $\vec{x}_f$ is one of the features of the sample $\vec{x}$ and *value* is a threshold value that was determined when building the tree. When a leaf node is reached, we return the value property of that leaf node as our prediction.

A few fundamental things to consider are
1. How easy will it be to build this tree?
2. How likely will the tree be balanced, thus increasing prediction performance?

For 1, building the tree will be fairly simple if we use partitioning. We will have to write logic for determining if we should add another node to the tree. We have three conditions under which this will happen.
1. The impurity of the dataset for the node is 0.
2. The maximum number of leaf nodes have already been created.
3. The height of the tree has reached the maximum value.
4. The dataset has less than three points.

For 2, this ultimately depends on how the given training data. In order to guarantee the tree is balanced, we would have to implement an AVL tree.

#### Algorithm Pseudo code

```
fit(X, y)
    // initialize a list to use as a queue
    node_queue = []
    
    // starting at the root node 
    node = BST.root_node
    
    // computing the impurity of the root node
    sse = y.shape[0] * VAR(y)
    
    // add the root node to the queue
    node_queue.enqueue({
        node: node
        depth: 0
        mask: All rows of X
        impurity: sse
        feature: 0
    })

    while node_queue is not empty:
        // dequeue a node
        node_dict = node_queue.dequeue()
        
        // leaf_size and max_height also have to not be None
        if BST.num_of_leaves >= leaf_size or node_dict.depth == max_height or node_dict.impurity == 0 or X has less than 3 rows
            set the node's value property to the mean of y
            continue to the next iteration
        
        find the best sample to create a split

        get the impurity of each split

        if BST.num_of_leaves + 1 <= leaf_size or leaf_size == None
            add a new node as the left child of node_dict.node
            enqueue a new node_dict to node_queue

        if BST.num_of_leaves + 1 <= leaf_size or leaf_size == None
            add a new node as the right child of node_dict.node
            enqueue a new node_dict to node_queue
        

```

## Tree Structure and Algorithms

As stated earlier, we want to use a binary search tree (BST) where each node either has a condition property, or a value property.
- The condition property is used to determine which path we should take when traversing down the tree (making a prediction). This property is only given to nodes that are not leaf nodes.
- The value property is the value we return for our prediction. Only leaf nodes will have this property.

We also want our BST to track it's height and the number of leaf nodes. Or, and this is a more simple approach, we can traverse the tree to calculate these values each time we need them, which at worst cost $O(n)$ time.

What is the exact logic we need to determine if we should or shouldn't add a node for cases 2 and 3?

For case 2, once we get the current number of leaf nodes, we can determine if we can or cannot add a node based on the following condition. If the number of leaf nodes is less than the max leaf size, add another node. Otherwise, do not add another node (or stop building the tree). The full condition will look like: if the number of leaf nodes is less than the max leaf size, and the impurity of the data is not 0, then add another node. Otherwise, if the max leaf size has been reached, stop building the tree.

For case 3, the solution is to actually track each node's depth in the tree, which is an equivalent, but more useful measure.

## Understanding partitioning



The goal of using partitioning is to avoid splitting a copy of the entire dataset to each individual node while building the tree.

Let $X$ denote our dataset and let $\vec{x}$ denote a sample in $X$.

Assume we've found a sample $\vec{x}^{(s)} \in X$ to split the data. This means we want to split the data along at the sample $\vec{x}^{(s)} along the feature $f$. One way we could do this is to partition the dataset into two separate datasets $X_{l}$ and $X_{r}$.

The way we do this is by computing the following. For each sample in $X$, if $\vec{x}_{f} < \vec{x}^{(s)}_{f}$ then it goes into $X_{l}$, otherwise, it goes into $X_{r}$. Where this becomes efficient is we can simply store the
indices for these samples instead of creating copies of the original dataset. For numpy, the easier way is to create a boolean mask for the rows we want in each partition.

The following code is an example of how this works.

In [19]:
# creating an array of random integers
X = np.random.default_rng(5).integers(20, size=(4, 4))
X

array([[13, 16,  0, 16],
       [ 9, 10, 12,  5],
       [19,  1,  5,  7],
       [11,  8,  2,  0]])

In [41]:
# let's say we want to partition the data along the 2nd feature
# and the sample we are splitting the data at is the second sample
f = 1 # second feature
split_value = X[1, f] # second sample (second row)
# old boolean mask way which has a small issue
mask = X[:, f] < split_value
print(split_value)
# mask = np.where(X[:, f] < split_value)
print(mask)
X_l = X[mask]
X_l

10
[False False  True  True]


array([[19,  1,  5,  7],
       [11,  8,  2,  0]])

A massive issue with this approach is it doesn't translate well for multiple splits. That is, how are we supposed to properly calculate the dataset for the node which uses the dataset $X_l$? Our algorithm always uses these splits to obtain the node's dataset from the original dataset. So if the mask generate for the split is relative to $X_l$, it won't work for parsing the next node's dataset from $X$.

The way we can fix this is we recalculate the split relative to the original dataset, and only include rows that are in the node's local dataset.

In [42]:
# this code snippet essentially solves the problem (hopefully)
# X_ll = (X[:, 3] < 7) & (X_l[:, None] == X).all(axis=2).any(axis=0)
X_ll = (X[:, 3] < 7) & mask
X_ll

array([False, False, False,  True])

## Determining the best split

To determine the best split, we will simply use a combination of a function that calculates the sum of squares for each possible split of the data and numpy's `argmin()` function.

First, we start off with a dataset $X$. Then for each sample $\vec{x} \in X$, we split the data into two subsets $X_l$ and $X_r$ using our partitioning method. Then, we compute the sum
$$
\sum_{i=1}^{n_{l}}{ (y_{i} - \bar{y}_{l})^{2} } + \sum_{j=1}^{n_{r}}{ (y_{j} - \bar{y}_{r})^{2} }
$$
and use `argmin()` to get the index of the sample which minimizes this sum.

In [22]:
# creating a test dataset
X = np.random.default_rng(5).integers(20, size=(4, 4))
y = np.random.default_rng(5).uniform(size=(4,))
print(X)
print(y)

# split the dataset into two disjoint subsets using X[1, 1] (second feature)
left_bool_mask = X[:, 1] < X[1, 1]
X_l = X[left_bool_mask]

right_bool_mask = X[:, 1] > X[1, 1]
X_r = X[right_bool_mask]

print(X_l)
print(X_l.size)
print(X_r)

[[13 16  0 16]
 [ 9 10 12  5]
 [19  1  5  7]
 [11  8  2  0]]
[0.80500292 0.80794079 0.51532556 0.28580138]
[[19  1  5  7]
 [11  8  2  0]]
8
[[13 16  0 16]]


In [23]:
# compute the sum of squares for each split
left_sum = X_l.shape[0] * np.var(y[left_bool_mask])
right_sum = X_r.shape[0] * np.var(y[right_bool_mask])
left_sum + right_sum

0.02634067482130236

In [24]:
# generalizing

def split_data(X, sample_idx, feature_idx):
    left_bool_mask = X[:, feature_idx] < X[sample_idx, feature_idx]
    right_bool_mask = X[:, feature_idx] > X[sample_idx, feature_idx]
    return left_bool_mask, right_bool_mask

def sum_of_squares(y):
    return y.shape[0] * np.std(y)

def get_best_split(X, y, feature_idx):
    left_bool_mask, right_bool_mask = split_data(X, 1, feature_idx)
    best_sample_idx = 1
    # best sum of squares error
    best_sse = sum_of_squares(y[left_bool_mask]) + sum_of_squares(y[right_bool_mask])
    for sample_idx in range(0, X.shape[0] - 1):
        left_bool_mask, right_bool_mask = split_data(X, sample_idx, feature_idx)
        sse = sum_of_squares(y[left_bool_mask]) + \
              sum_of_squares(y[right_bool_mask])
        
        if sse < best_sse:
            best_sample_idx = sample_idx
            best_sse = sse
    
    return best_sample_idx
        

print(get_best_split(X, y, 1))

1


/home/xavier/miniconda3/lib/python3.12/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/xavier/miniconda3/lib/python3.12/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/home/xavier/miniconda3/lib/python3.12/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


**Edge cases**
Sometimes, we don't want to compute a split. This happens in three the cases mentioned in *Our Algorithm*.

To handle the first case, we should modify `get_best_split()` to also return the best impurity measure. Then, the algorithm will check if it is zero, and will not add any other nodes to the queue.

To handle cases 2 and 4, we can create a function which checks the state of the tree (height and number of leaves), and then returns a boolean value to indicate if we can afford to add another node to the tree.

To handle case 4, we can simply check the size of the dataset before calling `get_best_split()`. If it's too small, then we just compute the mean and use that as our node's value.